In [29]:
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SequentialFeatureSelector

from plotly.offline import init_notebook_mode, plot
import plotly.express as px
import pandas as pd

from datasets import diabetes_data

In [30]:
original_X, original_y, train_X, train_y, test_X, test_y = diabetes_data()

# Forward selection

## Use 5-fold cross validation to estimate best number of features

In [34]:
init_notebook_mode(connected=True)

r2_means = []
number_of_features = original_X.shape[1]

# try every possible number of features
for i in range(1, number_of_features):
    selection_forward = SequentialFeatureSelector(
        LinearRegression(),
        n_features_to_select=i,
        direction='forward').fit(original_X, original_y)

    selected_features = original_X.columns[selection_forward.get_support()]
    r2_means.append(cross_val_score(LinearRegression(), original_X[selected_features], original_y).mean())

result = pd.DataFrame(zip(range(1,number_of_features), r2_means), columns=['n of features', 'R^2 (mean)'])
px.line(result, x='n of features', y='R^2 (mean)')

## Select features

In [19]:
selection_forward = SequentialFeatureSelector(LinearRegression(), n_features_to_select=6,
                                        direction='forward').fit(original_X, original_y)

In [20]:
selected_features = train_X.columns[selection_forward.get_support()]
selected_features

Index(['sex', 'bmi', 'bp', 's1', 's3', 's5'], dtype='object')

## Fit Linear Regression

In [21]:
# add constant, since statsmodels does not add it by default
original_X_subset = original_X[selected_features]
original_X_const = sm.add_constant(original_X_subset)
model = sm.OLS(original_y, original_X_const)

# fit model
result = model.fit()

In [22]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.513
Model:                            OLS   Adj. R-squared:                  0.507
Method:                 Least Squares   F-statistic:                     76.44
Date:                Tue, 15 Jun 2021   Prob (F-statistic):           6.31e-65
Time:                        16:02:39   Log-Likelihood:                -2388.1
No. Observations:                 442   AIC:                             4790.
Df Residuals:                     435   BIC:                             4819.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        152.1335      2.576     59.058      0.000     147.071     157.196
sex         -226.4941     60.430     -3.748      0.000    -345.265    -107.723
bmi          537.5727     65.430      8.216      0.000     408.974     666.172
bp           328.2573     62.869      5.221      0.000     204.693     451.822
s1          -136.6426     67.521     -2.024      0.044    -269.351      -3.935
s3          -240.6453     69.661     -3.455      0.001    -377.559    -103.731
s5           555.6131     76.803      7.234      0.000     404.662     706.564
==============================================================================
Omnibus:                        1.259   Durbin-Watson:                   2.035
Prob(Omnibus):                  0.533   Jarque-Bera (JB):                1.250
Skew:                           0.042   Prob(JB):                        0.535
Kurtosis:                       2.753   Cond. No.                         39.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# Backward selection

## Use 5-fold cross validation to estimate best number of features

In [23]:
r2_means = []
number_of_features = original_X.shape[1]

# try every possible number of features
for i in range(1, number_of_features):
    selection_forward = SequentialFeatureSelector(
        LinearRegression(),
        n_features_to_select=i,
        direction='backward').fit(original_X, original_y)

    selected_features = original_X.columns[selection_forward.get_support()]
    r2_means.append(cross_val_score(LinearRegression(), original_X[selected_features], original_y).mean())

result = pd.DataFrame(zip(range(1,number_of_features), r2_means), columns=['n of features', 'R^2 (mean)'])
px.line(result, x='n of features', y='R^2 (mean)')

## Select features

In [24]:
selection_backward = SequentialFeatureSelector(LinearRegression(), n_features_to_select=6,
                                        direction='backward').fit(original_X, original_y)

In [25]:
selected_features = train_X.columns[selection_backward.get_support()]
selected_features

Index(['sex', 'bmi', 'bp', 's1', 's2', 's5'], dtype='object')

## Fit Linear Regression

In [26]:
# add constant, since statsmodels does not add it by default
original_X_subset = original_X[selected_features]
original_X_const = sm.add_constant(original_X_subset)
model = sm.OLS(original_y, original_X_const)

# fit model
result = model.fit()

In [27]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.515
Model:                            OLS   Adj. R-squared:                  0.508
Method:                 Least Squares   F-statistic:                     76.95
Date:                Tue, 15 Jun 2021   Prob (F-statistic):           3.01e-65
Time:                        16:02:42   Log-Likelihood:                -2387.3
No. Observations:                 442   AIC:                             4789.
Df Residuals:                     435   BIC:                             4817.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        152.1335      2.572     59.159      0.000     147.079     157.188
sex         -226.5106     59.857     -3.784      0.000    -344.155    -108.866
bmi          529.8730     65.620      8.075      0.000     400.901     658.845
bp           327.2198     62.693      5.219      0.000     204.001     450.439
s1          -757.9379    160.435     -4.724      0.000   -1073.262    -442.614
s2           538.5859    146.738      3.670      0.000     250.182     826.989
s5           804.1923     80.173     10.031      0.000     646.617     961.767
==============================================================================
Omnibus:                        1.187   Durbin-Watson:                   2.043
Prob(Omnibus):                  0.552   Jarque-Bera (JB):                1.172
Skew:                           0.016   Prob(JB):                        0.557
Kurtosis:                       2.750   Cond. No.                         85.7
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""